In [196]:
import pandas as pd
from sklearn import preprocessing
import sklearn.model_selection as ms
from sklearn import linear_model
import sklearn.metrics as sklm
import numpy as np
import numpy.random as nr
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as ss
import math


pd.options.display.max_columns = None
pd.set_option('display.max_colwidth', -1)
pd.set_option('mode.chained_assignment', None)
pd.set_option('max_colwidth', 800)

WORK_CUST = './AdvWorksCusts.csv'
AVE_SPEND = './AW_AveMonthSpend.csv'
BIKE_BUYER = './AW_BikeBuyer.csv'
TEST = './AW_test.csv'


In [197]:
work_cust_df = pd.read_csv(WORK_CUST)
ave_spend_df = pd.read_csv(AVE_SPEND)
bike_buyer_df = pd.read_csv(BIKE_BUYER)
val = pd.read_csv(TEST)

LABEL = 'AveMonthSpend'

In [198]:
#Keep the last record for each unique customerID
work_cust_df = work_cust_df.drop_duplicates(subset='CustomerID', keep='last')
ave_spend_df = ave_spend_df.drop_duplicates(subset='CustomerID', keep='last')
bike_buyer_df = bike_buyer_df.drop_duplicates(subset='CustomerID',keep='last')

# Joing 3 dataframes into one DF
all_df = pd.merge(work_cust_df, ave_spend_df, left_on="CustomerID", right_on="CustomerID", how='left')
all_df = pd.merge(all_df, bike_buyer_df, left_on="CustomerID", right_on="CustomerID", how='left')

In [199]:
all_df.head()

,CustomerID,Title,FirstName,MiddleName,LastName,Suffix,AddressLine1,AddressLine2,City,StateProvinceName,CountryRegionName,PostalCode,PhoneNumber,BirthDate,Education,Occupation,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome,AveMonthSpend,BikeBuyer
0,11000,NaN,Jon,V,Yang,NaN,3761 N. 14th St,NaN,Rockhampton,Queensland,Australia,4700,1 (11) 500 555-0162,1966-04-08,Bachelors,Professional,M,M,1,0,0,2,137947,89,0
1,11001,NaN,Eugene,L,Huang,NaN,2243 W St.,NaN,Seaford,Victoria,Australia,3198,1 (11) 500 555-0110,1965-05-14,Bachelors,Professional,M,S,0,1,3,3,101141,117,1
2,11002,NaN,Ruben,NaN,Torres,NaN,5844 Linden Land,NaN,Hobart,Tasmania,Australia,7001,1 (11) 500 555-0184,1965-08-12,Bachelors,Professional,M,M,1,1,3,3,91945,123,0
3,11003,NaN,Christy,NaN,Zhu,NaN,1825 Village Pl.,NaN,North Ryde,New South Wales,Australia,2113,1 (11) 500 555-0162,1968-02-15,Bachelors,Professional,F,S,0,1,0,0,86688,50,0
4,11004,NaN,Elizabeth,NaN,Johnson,NaN,7553 Harness Circle,NaN,Wollongong,New South Wales,Australia,2500,1 (11) 500 555-0131,1968-08-08,Bachelors,Professional,F,S,1,4,5,5,92771,95,1


In [200]:
keep_cols = ['CustomerID', 'CountryRegionName', 'BirthDate', 
             'Education', 'Occupation',
             'Gender', 'MaritalStatus', 'HomeOwnerFlag', 'NumberCarsOwned',
             'NumberChildrenAtHome', 'TotalChildren', 
             'YearlyIncome', 'AveMonthSpend']

keep_cols_val = ['CustomerID', 'CountryRegionName', 'BirthDate', 
             'Education', 'Occupation',
             'Gender', 'MaritalStatus', 'HomeOwnerFlag', 'NumberCarsOwned',
             'NumberChildrenAtHome', 'TotalChildren', 
             'YearlyIncome']

In [201]:
all_df = all_df.sample(frac=0.15, replace=True)


In [202]:
df = all_df[keep_cols]
val = val[keep_cols_val]

In [203]:
from datetime import datetime
from datetime import date


def calculate_age(born):
    try:
        born = datetime.strptime(born, "%Y-%m-%d").date()
        particular_date = datetime(1998, 1, 1)
        return particular_date.year - born.year - ((particular_date.month, particular_date.day) < (born.month, born.day))
    except:
        return 0

    
def calculate_age_val(born):
    try:
        born = datetime.strptime(born, "%d/%m/%Y").date()
        particular_date = datetime(1998, 1, 1)
        return particular_date.year - born.year - ((particular_date.month, particular_date.day) < (born.month, born.day))
    except:
        return 0   
    
    
def age_cat(age):
    if age < 25:
        return 'under 25'
    elif age >= 25 and age <= 45:
        
        return 'between_25_and_45'
    
    elif age > 45 and age <= 55:
        return 'between_44_and_55'
    
    elif age > 55:
        return 'over_55'


In [204]:
df.head()

,CustomerID,CountryRegionName,BirthDate,Education,Occupation,Gender,MaritalStatus,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome,AveMonthSpend
2517,13855,United States,1973-08-06,Graduate Degree,Skilled Manual,F,S,1,0,0,1,57048,49
2004,13265,Germany,1956-01-22,High School,Professional,M,S,1,4,4,4,143605,134
4101,15647,Australia,1978-11-20,Bachelors,Clerical,F,M,1,0,0,0,26667,40
8131,20180,United States,1962-07-03,Partial College,Skilled Manual,F,M,1,1,0,1,72286,54
3857,15361,United States,1933-05-18,Graduate Degree,Management,M,S,0,4,0,0,180112,72


In [205]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from pandas.api.types import is_numeric_dtype

from sklearn import preprocessing

def preprocess(tmp_df):
    
    # Make new columns
    tmp_df['age'] = tmp_df['BirthDate'].apply(calculate_age)    
    
#     tmp_df['age_cat'] = tmp_df['age'].apply(age_cat)
    tmp_df = tmp_df.drop('BirthDate',axis = 1)
    
    
#     #One hot encode categorical features
#     tmp_df = pd.concat([tmp_df,pd.get_dummies(tmp_df['age_cat'], prefix='category')],axis=1)
#     tmp_df.drop(['age_cat'],axis=1, inplace=True)

    tmp_df = pd.concat([tmp_df,pd.get_dummies(tmp_df['CountryRegionName'], prefix='country')],axis=1)
    tmp_df.drop(['CountryRegionName'],axis=1, inplace=True)    
    
    tmp_df = pd.concat([tmp_df,pd.get_dummies(tmp_df['Education'], prefix='education')],axis=1)
    tmp_df.drop(['Education'],axis=1, inplace=True)  
    
    tmp_df = pd.concat([tmp_df,pd.get_dummies(tmp_df['Occupation'], prefix='job')],axis=1)
    tmp_df.drop(['Occupation'],axis=1, inplace=True)
    
    tmp_df = pd.concat([tmp_df,pd.get_dummies(tmp_df['MaritalStatus'], prefix='job')],axis=1)
    tmp_df.drop(['MaritalStatus'],axis=1, inplace=True)  
    
    tmp_df['Gender'] = tmp_df['Gender'].apply(lambda x: 'male' if x == 'M' else 'female')
    tmp_df = pd.concat([tmp_df,pd.get_dummies(tmp_df['Gender'], prefix='gender')],axis=1)
    tmp_df.drop(['Gender'],axis=1, inplace=True)   
        
    
    # Scale Number Features 
    tmp_df = tmp_df[tmp_df.YearlyIncome != 'Professional']
    tmp_df['YearlyIncome'] = tmp_df['YearlyIncome'].values.astype(float)
    scaler = preprocessing.StandardScaler().fit(tmp_df[['YearlyIncome']])
    tmp_df['YearlyIncome'] = scaler.transform(tmp_df[['YearlyIncome']])
    
    
    tmp_df['age'] = tmp_df['age'].values.astype(float)
    scaler = preprocessing.StandardScaler().fit(tmp_df[['age']])
    tmp_df['age'] = scaler.transform(tmp_df[['age']])
    
    
    # Drop Customer ID 
    tmp_df.drop(['CustomerID'],axis=1, inplace=True)

    print(tmp_df.shape)
                               
    return tmp_df

In [206]:
df = preprocess(df)
val = preprocess(val)

(2461, 27)
(500, 26)


/anaconda3/lib/python3.6/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)
/anaconda3/lib/python3.6/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


In [207]:
df.columns.values

array(['HomeOwnerFlag', 'NumberCarsOwned', 'NumberChildrenAtHome',
       'TotalChildren', 'YearlyIncome', 'AveMonthSpend', 'age',
       'country_Australia', 'country_Canada', 'country_France',
       'country_Germany', 'country_United Kingdom',
       'country_United States', 'education_Bachelors ',
       'education_Graduate Degree', 'education_High School',
       'education_Partial College', 'education_Partial High School',
       'job_Clerical', 'job_Management', 'job_Manual', 'job_Professional',
       'job_Skilled Manual', 'job_M', 'job_S', 'gender_female',
       'gender_male'], dtype=object)

In [209]:
feature_cols = ['HomeOwnerFlag', 'NumberCarsOwned',
               'NumberChildrenAtHome', 'TotalChildren', 'YearlyIncome',
                'age', 'country_Australia', 'country_Canada',
               'country_France', 'country_Germany', 'country_United Kingdom',
               'country_United States', 'education_Bachelors ',
               'education_Graduate Degree', 'education_High School',
               'education_Partial College', 'education_Partial High School',
               'job_Clerical', 'job_Management', 'job_Manual', 'job_Professional',
               'job_Skilled Manual', 'job_M', 'job_S', 'gender_female',
               'gender_male']

In [210]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2461 entries, 2517 to 5810
Data columns (total 27 columns):
HomeOwnerFlag                    2461 non-null int64
NumberCarsOwned                  2461 non-null int64
NumberChildrenAtHome             2461 non-null int64
TotalChildren                    2461 non-null int64
YearlyIncome                     2461 non-null float64
AveMonthSpend                    2461 non-null int64
age                              2461 non-null float64
country_Australia                2461 non-null uint8
country_Canada                   2461 non-null uint8
country_France                   2461 non-null uint8
country_Germany                  2461 non-null uint8
country_United Kingdom           2461 non-null uint8
country_United States            2461 non-null uint8
education_Bachelors              2461 non-null uint8
education_Graduate Degree        2461 non-null uint8
education_High School            2461 non-null uint8
education_Partial College        2461 

In [211]:
val.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 500 entries, 0 to 499
Data columns (total 26 columns):
HomeOwnerFlag                    500 non-null int64
NumberCarsOwned                  500 non-null int64
NumberChildrenAtHome             500 non-null int64
TotalChildren                    500 non-null int64
YearlyIncome                     500 non-null float64
age                              500 non-null float64
country_Australia                500 non-null uint8
country_Canada                   500 non-null uint8
country_France                   500 non-null uint8
country_Germany                  500 non-null uint8
country_United Kingdom           500 non-null uint8
country_United States            500 non-null uint8
education_Bachelors              500 non-null uint8
education_Graduate Degree        500 non-null uint8
education_High School            500 non-null uint8
education_Partial College        500 non-null uint8
education_Partial High School    500 non-null uint8
job_Cle

In [212]:
from sklearn.model_selection import train_test_split

X = df[feature_cols] 
y = df[LABEL]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

In [213]:
X_train[:5]

,HomeOwnerFlag,NumberCarsOwned,NumberChildrenAtHome,TotalChildren,YearlyIncome,age,country_Australia,country_Canada,country_France,country_Germany,country_United Kingdom,country_United States,education_Bachelors,education_Graduate Degree,education_High School,education_Partial College,education_Partial High School,job_Clerical,job_Management,job_Manual,job_Professional,job_Skilled Manual,job_M,job_S,gender_female,gender_male
14551,0,2,1,2,-0.837098,1.474642,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,1,0,1
11142,1,3,4,4,1.368595,0.410444,0,0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,1,1,0
4752,1,0,0,4,0.623350,0.233077,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,1,0,0,1
12799,1,0,0,0,-0.518194,-0.565072,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,1,1,0
9270,1,2,1,2,0.407350,0.942543,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,0,1,0,0,1


#### Modelling

In [214]:
from sklearn import linear_model


## define and fit the linear regression model
lin_mod = linear_model.LinearRegression(fit_intercept = False)
lin_mod.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=False, n_jobs=None, normalize=False)

In [215]:
print(lin_mod.intercept_)
print(lin_mod.coef_)

0.0
[ 0.04021042  0.11580115 10.58896522  0.36224002  8.13310436 -1.08296144
  6.07023892  6.75302563  6.02182908  6.71902867  6.62868157  6.29859648
  8.65950528  7.50878814  7.47385215  7.85054073  6.99871405  8.50721833
  5.16343839  8.18702997  7.61675484  9.01695881 22.25644724 16.23495312
  5.43687368 33.05452667]


In [216]:
%matplotlib inline
def print_metrics(y_true, y_predicted, n_parameters):
    ## First compute R^2 and the adjusted R^2
    r2 = sklm.r2_score(y_true, y_predicted)
    r2_adj = r2 - (n_parameters - 1)/(y_true.shape[0] - n_parameters) * (1 - r2)

    ## Print the usual metrics and the R^2 values
    print('Mean Square Error      = ' + str(sklm.mean_squared_error(y_true, y_predicted)))
    print('Root Mean Square Error = ' + str(math.sqrt(sklm.mean_squared_error(y_true, y_predicted))))
    print('Mean Absolute Error    = ' + str(sklm.mean_absolute_error(y_true, y_predicted)))
    print('Median Absolute Error  = ' + str(sklm.median_absolute_error(y_true, y_predicted)))
    print('R^2                    = ' + str(r2))
    print('Adjusted R^2           = ' + str(r2_adj))


y_score = lin_mod.predict(X_test) 
print_metrics(y_test, y_score, 1)    

Mean Square Error      = 40.37731412675399
Root Mean Square Error = 6.35431460715898
Mean Absolute Error    = 4.691417812458821
Median Absolute Error  = 3.512797953094193
R^2                    = 0.9405748357771785
Adjusted R^2           = 0.9405748357771785


In [217]:
y_score[:10]

array([107.68141171,  64.8377363 ,  39.18963214,  94.9756254 ,
        47.81621538,  40.36363998,  33.60706814,  47.81330038,
        78.1214623 , 101.26579145])

In [218]:
preds = lin_mod.predict(val)

In [220]:
for p in preds:
    print(p)

44.8652618027726
106.31216489450199
52.83140842343929
88.71514775144627
59.42737285720408
43.38778856894162
95.79739394024932
125.38123221354286
100.77677142886816
56.075986380450665
57.9064631677789
50.44909067190043
73.15149875983727
47.08617599249485
36.00047257803227
50.089375182769764
86.00932261255969
72.90673934382525
108.12880540168834
58.39221644790839
67.28318652098764
76.01191070455198
144.54008039127876
84.27781690969218
53.417772331250625
78.18001353215236
85.76842562363842
114.28430818955988
75.66424041629114
61.126783372235046
68.49293557705747
78.27501967376838
42.3041770240425
73.53527975975814
104.27621436605334
100.94244085554098
144.62547058162804
91.20264693474192
56.242257560678986
86.2291152477442
45.21343298177215
80.48567114953696
80.19437920141587
48.34800408529224
57.81574597874832
74.93947292722564
60.27327357598357
86.77854882764129
114.41447691441155
80.64743259314258
84.22316057802166
93.81023473263126
80.48415174788673
66.38352198136224
46.47082228122733